<h2 align = "center">Mod 1: Change the Network Structure</h2>

Focus on hidden layers and the number of hidden layers.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import latte
import latte.nn as nn
import latte.optim as optim
import latte.functional as F
import latte.utils.data as data

import lattevision as lv
import lattevision.datasets as dsets
import lattevision.transforms as T


In [2]:
mnist_root = '../data/mnist'
mnist_transform = T.Compose([T.ToTensor(), T.Normalize((0.1307,), (0.3081,))])

mnist_train = dsets.MNIST(mnist_root, train=True, transform=mnist_transform)
mnist_test = dsets.MNIST(mnist_root, train=False, transform=mnist_transform)

train_batch_size = 128
test_batch_size = 512

train_loader = data.DataLoader(mnist_train, batch_size=train_batch_size, shuffle=True)
test_loader = data.DataLoader(mnist_test, batch_size=test_batch_size, shuffle=False)


In [3]:
for n_hidden in [64, 128, 256, 512]:
    print(f'Network Structure: [784, {n_hidden}, 10]')

    class Model(nn.Module):
        def __init__(self) -> None:
            super().__init__()
            self.fc1 = nn.Linear(784, 256)
            self.relu = nn.ReLU()
            self.fc2 = nn.Linear(256, 10)

        def forward(self, x):
            out = self.fc1(x)
            out = self.relu(out)
            out = self.fc2(out)
            return out

    model = Model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    max_epochs = 15

    test_accuracies = []

    for epoch in range(max_epochs):
        # Training
        for ii, (data, target) in enumerate(train_loader):
            batch_size = data.shape[0]
            if batch_size != train_batch_size:
                continue

            data = latte.Tensor(data.reshape(batch_size, -1))
            target = latte.Tensor(target)

            # Forward pass
            optimizer.zero_grad()
            output = model(data)

            # Compute loss
            loss = criterion(output, target)

            # Backward pass
            loss.backward()
            optimizer.step()

        # Test
        correct = 0
        total = 0
        for data, target in test_loader:
            batch_size = data.shape[0]
            if batch_size != test_batch_size:
                continue

            data = latte.Tensor(data.reshape(batch_size, -1))
            target = latte.Tensor(target)

            output = model(data)
            correct += (output.data.argmax(axis=1) == target.data).sum()
            total += target.shape[0]

        accuracy = correct / total
        test_accuracies.append(np.round(accuracy, 4))

    print(f'\tAll: {test_accuracies}')
    print(f'\tFinal: {np.max(test_accuracies): .4f}')


Network Structure: [784, 64, 10]
	All: [0.9115, 0.9313, 0.9442, 0.9552, 0.9615, 0.9688, 0.9703, 0.9718, 0.973, 0.9744, 0.974, 0.9753, 0.9762, 0.9767, 0.9767]
	Final:  0.9767
Network Structure: [784, 128, 10]
	All: [0.9069, 0.9314, 0.9444, 0.9556, 0.9646, 0.9657, 0.9695, 0.972, 0.9738, 0.9741, 0.9738, 0.975, 0.9756, 0.9738, 0.9754]
	Final:  0.9756
Network Structure: [784, 256, 10]
	All: [0.9074, 0.9295, 0.9423, 0.9571, 0.9607, 0.9664, 0.9696, 0.9706, 0.9736, 0.9743, 0.9734, 0.9736, 0.9734, 0.9758, 0.9744]
	Final:  0.9758
Network Structure: [784, 512, 10]
	All: [0.907, 0.9298, 0.9451, 0.9539, 0.9628, 0.9637, 0.9686, 0.9707, 0.9708, 0.9718, 0.9725, 0.9738, 0.9735, 0.9737, 0.9731]
	Final:  0.9738


In [6]:
class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out


model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
max_epochs = 15

test_accuracies = []

for epoch in range(max_epochs):
    # Training
    for ii, (data, target) in enumerate(train_loader):
        batch_size = data.shape[0]
        if batch_size != train_batch_size:
            continue

        data = latte.Tensor(data.reshape(batch_size, -1))
        target = latte.Tensor(target)

        # Forward pass
        optimizer.zero_grad()
        output = model(data)

        # Compute loss
        loss = criterion(output, target)

        # Backward pass
        loss.backward()
        optimizer.step()

    # Test
    correct = 0
    total = 0
    for data, target in test_loader:
        batch_size = data.shape[0]
        if batch_size != test_batch_size:
            continue

        data = latte.Tensor(data.reshape(batch_size, -1))
        target = latte.Tensor(target)

        output = model(data)
        correct += (output.data.argmax(axis=1) == target.data).sum()
        total += target.shape[0]

    accuracy = correct / total
    test_accuracies.append(np.round(accuracy, 4))

print(f'\tAll: {test_accuracies}')
print(f'\tFinal: {np.max(test_accuracies): .4f}')


	All: [0.5279, 0.8723, 0.9109, 0.9306, 0.9438, 0.9527, 0.9586, 0.9661, 0.965, 0.967, 0.9698, 0.9719, 0.9733, 0.9734, 0.9741]
	Final:  0.9741
